## Ejercicio 6: Convertir descripciones en lenguaje natural a consultas

Los LLMs son excelentes traductores de lenguaje natural a código. Incluso si no conocemos un lenguaje, podemos pedirle que nos genere el código correspondiente. De este modo, en vez de perder tiempo en un ensayo y error para llegar a escrbir el código, podemos centrarnos en analizar los resultados del mismo

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

#print(type(OPENAI_API_KEY),OPENAI_API_KEY)
#print(type(API_VERSION),API_VERSION)
#print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [2]:
from openai import AzureOpenAI

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

##### Escribimos lo que queremos obtener con lenguaje natural

In [3]:
descripcion = (
    "Quiero obtener la suma de las calorias realizadas por cada id en el último mes, "
    "ordenadas de mayor a menor."
)

##### Y ahora creamos el prompt correspondiente

In [4]:
import pandas as pd
data = {
  "id" : [1,2,3],
  "calories": [420, 380, 390],
  "duration": [50, 40, 45]
}

#load data into a DataFrame object:
df = pd.DataFrame(data)

In [5]:
prompt_sql = (
    f"Convierte la siguiente descripción en una consulta SQL:\n\"\"\"\n{descripcion}\n\"\"\"\n"
    "Proporciona únicamente el código SQL, sin texto adicional. Razona paso a paso y asegúrate de que el código es correcto."
    f"te comparto un dataframe de pandas {df}"
)

##### Lo probamos
- Este ejemplo es con SQL, pero obviamente se puede usar para python, R o cualquier otro lenguaje de programacion

In [6]:
from IPython.display import Markdown

client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)

response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_sql}],
        temperature=0.3,
        max_tokens=800,
    )
consulta_sql = response.choices[0].message.content.strip()
print("\nConsulta SQL generada:")
display(Markdown(consulta_sql))


Consulta SQL generada:


```sql
SELECT 
    id, 
    SUM(calories) AS total_calories
FROM 
    your_table_name
WHERE 
    DATE_PART('month', CURRENT_DATE) - DATE_PART('month', your_date_column) = 1
GROUP BY 
    id
ORDER BY 
    total_calories DESC;
```

##### - ¿Cómo se traduce cada parte de la descripción en elementos de la consulta SQL (SELECT, WHERE, GROUP BY, etc.)?
##### - ¿Qué cambios harías al prompt si la consulta necesitara filtros adicionales o uniones entre tablas?